In [121]:
import adagram
import gensim
import pandas as pd
import nltk
from lxml import html
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.wsd import lesk
from pymorphy2 import MorphAnalyzer
from string import punctuation
import json, os
from collections import Counter
import numpy as np
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from matplotlib import pyplot as plt
import matplotlib
import warnings
from nltk.tokenize import sent_tokenize
import random

warnings.filterwarnings('ignore')
morph = MorphAnalyzer()
punct = punctuation + '«»—…“”*№–'

stops = set(stopwords.words('english'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word]

    return words

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split() if word and word not in stops]
    words = [word for word in words if word]

    return words

corpus = open('C:\\NonSystem\\ChromeDownload\\corpus_eng.txt', 'r', encoding='utf-8')
corpus = corpus.read()
corpus = corpus.strip(punct)
corpus = sent_tokenize(corpus)
breaksent = []
for i in corpus:
    sent = i
    sent = sent.strip(punct)
    sent = sent.split(' ')
    if 'break' in sent:
        breaksent.append(i)        


Все Строчки, в которых есть слово 'break'.

In [122]:
breaksent

['They break the rules in order to convince the rule-makers that they need to change the rules, which is itself a kind of state-approved process.',
 'In the second innings, he got locked up in his position so much, intent on trying to play an offspinner against the turn, that he was stunned by a big-turning off break that ricocheted off his pads and fell on the stumps.',
 'She wasn’t very easy to break in.',
 'Their five-day schedule break is Feb. 10-15 and they have 13 of their final 20 games at home starting March 2.',
 '"Back in my poor college days when I worked at Walmart we had a fight break out over a bike.',
 'Since then, the party has said it respects the will of the people, but many Labour lawmakers are hoping to steer the talks with the European Union away from what some fear will be a clean break with the bloc\'s lucrative single market - the so-called "hard Brexit."',
 'or is it to break the record of first president to get indicted and cuffed while being sworn in?',
 '“Ye

Случайно выбраны 10 предложений.

In [123]:
chosensent = []

random = random.sample(range(0,len(breaksent)),10)
for i in random:
    chosensent.append(breaksent[i])

chosensent

['Wearing hats to block the midday New Mexico sun, the summer campers crouched in the rows of the 14-acre La Semilla Community Farm would soon break for lunch.',
 'Redknapp made a number of changes at the break as it was a friendly and an injury meant they only had ten men.',
 '“I may have gray hair,” Alan says to break the ice, “but I self-identify as a millennial.” \nShe laughs.',
 "Davies said he thought it was possible Britain would get a transitional deal because the EU's other 27 countries realized their companies could also be damaged by being unable to raise finance in London in the event of a hard break with the EU.",
 'Trump\'s win heralds a hard break in a relationship that grew extremely close during the Cold War, before wobbling when Germany refused to go along with Bush\'s Iraq war and was derided by his Defence Secretary Donald Rumsfeld as "old Europe".',
 'Fiorentina continued to look threatening although Joao Mario missed a good chance for Inter on the break when he sh

In [132]:
word = 'break'
sen = []#Список всех определений слова break

for synset in wn.synsets(word):
    sen.append(synset.definition())


sense_match = {}#Нормализация определений
for sense in sense_counter:
    sense_norm = sense.split(' ')
    for word in sense_norm:
        word.strip('')
        if word in stops:
            sense_norm.remove(word)
    new_sense = []
    for word in sense_norm:
        new_word = word.strip(punct)
        if new_word not in stops:
            new_sense.append(new_word)
    sense_match[sense] = new_sense

sense_match

{'some abrupt occurrence that interrupts an ongoing activity': ['abrupt',
  'occurrence',
  'interrupts',
  'ongoing',
  'activity'],
 'an unexpected piece of good luck': ['unexpected', 'piece', 'good', 'luck'],
 "(geology) a crack in the earth's crust resulting from the displacement of one side with respect to the other": ['geology',
  'crack',
  "earth's",
  'crust',
  'resulting',
  'displacement',
  'one',
  'side',
  'respect'],
 'a personal or social separation (as between opposing factions)': ['personal',
  'social',
  'separation',
  'opposing',
  'factions'],
 'a pause from doing something (as work)': ['pause', 'something', 'work'],
 'the act of breaking something': ['act', 'breaking', 'something'],
 'a time interval during which there is a temporary cessation of something': ['time',
  'interval',
  'temporary',
  'cessation',
  'something'],
 'breaking of hard tissue such as bone': ['breaking',
  'hard',
  'tissue',
  'bone'],
 'the occurrence of breaking': ['occurrence', 'br

In [130]:
sent_match = {}#Нормализация предложений
for sent in chosensent:
    
    sent_match[sent] = []
    sent_words = normalize(sent)
    
    for word in sent_words:
        word = word.strip(punct)
        if word not in stops:
            if word != 'break':
                sent_match[sent].append(word)

sent_match
    
            

{'Wearing hats to block the midday New Mexico sun, the summer campers crouched in the rows of the 14-acre La Semilla Community Farm would soon break for lunch.': ['wearing',
  'hats',
  'block',
  'midday',
  'new',
  'mexico',
  'sun',
  'summer',
  'campers',
  'crouched',
  'rows',
  '14-acre',
  'la',
  'semilla',
  'community',
  'farm',
  'would',
  'soon',
  'lunch'],
 'Redknapp made a number of changes at the break as it was a friendly and an injury meant they only had ten men.': ['redknapp',
  'made',
  'number',
  'changes',
  'friendly',
  'injury',
  'meant',
  'ten',
  'men'],
 '“I may have gray hair,” Alan says to break the ice, “but I self-identify as a millennial.” \nShe laughs.': ['may',
  'gray',
  'hair',
  'alan',
  'says',
  'ice',
  'self-identify',
  'millennial',
  'laughs'],
 "Davies said he thought it was possible Britain would get a transitional deal because the EU's other 27 countries realized their companies could also be damaged by being unable to raise fi

Score for one definition = the sum of similarity scores between each word in the sentence and each word in the definition devided by the number of words in that definition.

In [133]:
import networkx as nx
from nltk.corpus import wordnet as wn

for n in range(0,len(chosensent)):
    current_sent = chosensent[n]
    score_counter = {}
    for i in range(0,len(sense_match)):
        score_group = []
        score = 0
        for word_sent in sent_match[current_sent]:
            for word_sense in sense_match[sen[i]]:
                try:
                    score += wn.lch_similarity(wn.synsets(word_sent)[0], wn.synsets(word_sense)[0])
                except:
                    score += 0
        score = score / len(sense_match[sen[i]])
        score_counter[sen[i]] = score
    print(current_sent)
    print(max(score_counter,key=score_counter.get))
    
    print('  ')
    
   
        
    


Wearing hats to block the midday New Mexico sun, the summer campers crouched in the rows of the 14-acre La Semilla Community Farm would soon break for lunch.
come to an end
  
Redknapp made a number of changes at the break as it was a friendly and an injury meant they only had ten men.
the occurrence of breaking
  
“I may have gray hair,” Alan says to break the ice, “but I self-identify as a millennial.” 
She laughs.
scatter or part
  
Davies said he thought it was possible Britain would get a transitional deal because the EU's other 27 countries realized their companies could also be damaged by being unable to raise finance in London in the event of a hard break with the EU.
the occurrence of breaking
  
Trump's win heralds a hard break in a relationship that grew extremely close during the Cold War, before wobbling when Germany refused to go along with Bush's Iraq war and was derided by his Defence Secretary Donald Rumsfeld as "old Europe".
the occurrence of breaking
  
Fiorentina co

Precision: 4/10